In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
from collections import defaultdict

In [2]:
# generate list of file paths for xml metadata
xml_directory = 'data/xml/'
xml_files = os.listdir(xml_directory)
xml_files = [xml_directory+file for file in xml_files]

In [3]:
def get_metadata(xml_file_path):

    tree = ET.parse(xml_file_path)# parse xml file
    article_meta = tree.getroot()[0][1] # get article metadata branch
    metadata = defaultdict(str) # create empty dict for metadata

    # get article IDs
    article_ids = article_meta.iter('article-id')
    for i in article_ids:
        metadata[i.attrib['pub-id-type']+'-id'] = i.text

    # get title
    for i in article_meta.iter('article-title'):
        metadata['title'] += i.text

    # get authors
    authors = []
    for contrib in article_meta.iter('contrib'):
        for string_name in contrib.findall('string-name'):
            surname = string_name.findtext('surname')
            given_names = string_name.findtext('given-names')
            name = f'{surname}, {given_names}'
            authors.append(name)
    metadata['authors'] = '; '.join(authors)

    # get year, month, volume, issue, page numbers
    metadata['year'] = article_meta.find('pub-date').find('year').text
    metadata['month'] = article_meta.find('pub-date').find('month').text
    metadata['volume'] = article_meta.find('volume').text
    try:
        metadata['issue'] = article_meta.find('issue').text
    except:
        metadata['issue'] = ''
    try:
        metadata['pages'] = str(article_meta.find('fpage').text) + "-" + str(article_meta.find('lpage').text)
    except:
        metadata['pages'] = ''

    # get keywords
    try:
        metadata['keywords'] = '; '.join([kwd.text for kwd in article_meta.find('kwd-group').findall('kwd')])
    except AttributeError:
        metadata['keywords'] = ''

    # get abstract
    abstract = article_meta.iter('abstract')
    for a in abstract:
        for i in a:
            metadata['abstract'] += i.text + " "

    return pd.Series(metadata)

In [7]:
df = pd.DataFrame([get_metadata(file) for file in xml_files])
df.head(5)

,jstor-id,title,authors,year,month,volume,issue,pages,keywords,abstract,pub-doi-id
0,1353809,Volume Information,,1999,12,266,1437,2553-xvii,,NaN,NaN
1,1353810,Front Matter,,1999,12,266,1437,None-None,,NaN,NaN
2,1353811,Knowing No Fear,"Sprengelmeyer, Reiner; Young, Andrew W.; Schro...",1999,12,266,1437,2451-2456,Amygdala; Facial expression; Emotion recogniti...,People with brain injuries involving the amygd...,NaN
3,1353812,Anti-Predator Behaviour Changes Following an A...,"Díaz-Uriarte, Ramón",1999,12,266,1437,2457-2464,Anti-predator behaviour; Predation risk; Aggre...,Avoiding predators may conflict with territori...,NaN
4,1353813,Regulation of Queen-Worker Conflict in Bumble-...,"Bloch, Guy",1999,12,266,1437,2465-2469,Social evolution; Bumble-bees; Reproduction; Q...,In annual colonies of bumble-bees overt queen-...,NaN


### explore metadata to check for errors

In [22]:
# check for duplicate jstor-ids:
df['jstor-id'].value_counts(dropna=False).value_counts()

1    16842
Name: jstor-id, dtype: int64

All rows have a unique, non-null jstor-id

In [23]:
# check for duplicate pub-doi-ids
df['pub-doi-id'].value_counts(dropna=False).value_counts()

1        2170
2          12
14648       1
Name: pub-doi-id, dtype: int64

In [24]:
# some doi-ids are duplicated - investigate
doi_notnulls = df[df['pub-doi-id'].notnull()]
doi_notnulls[doi_notnulls['pub-doi-id'].duplicated(keep=False)].sort_values('pub-doi-id')

,jstor-id,title,authors,year,month,volume,issue,pages,keywords,abstract,pub-doi-id
3063,30047881,Errata: Genetic Variation for an Aphid Wing Po...,"Braendle, Christian; Friebe, Ilvy; Caillaud, M...",2005,12,272,1581,2659-2659,,NaN,10.1098/rspb.2004.2995
3080,30047899,Genetic Variation for an Aphid Wing Polyphenis...,"Braendle, Christian; Friebe, Ilvy; Caillaud, M...",2005,3,272,1563,657-664,Acyrthosiphon pisum; alternative phenotypes; a...,Many polyphenisms are examples of adaptive phe...,10.1098/rspb.2004.2995
2868,30047675,Supply-Side Invasion Ecology: Characterizing P...,"Verling, Emma; Ruiz, Gregory M.; Smith, L. Dav...",2005,6,272,1569,1249-1256,biological invasion; ballast water; propagule ...,The observed rates and deleterious impacts of ...,10.1098/rspb.2005.3090
3064,30047882,Errata: Supply-Side Invasion Ecology: Characte...,"Verling, Emma; Ruiz, Gregory M.; Smith, L. Dav...",2005,12,272,1581,2659-2659,,NaN,10.1098/rspb.2005.3090
3015,30047831,Cattle Domestication in the Near East Was Foll...,"Götherström, Anders; Anderung, Cecilia; Hellbo...",2005,11,272,1579,2345-2350,cattle; wild ox; domestication; Y chromosome; ...,Domesticated cattle were one of the cornerston...,10.1098/rspb.2005.3243
3065,30047883,Errata: Cattle Domestication in the near East ...,"Götherstöm, Anders; Anderung, Cecilia; Hellbor...",2005,12,272,1581,2660-2660,,NaN,10.1098/rspb.2005.3243
913,25223556,Editorial,"Hill, Bill",2006,8,273,1597,1985-1985,,NaN,10.1098/rspb.2006.3652
943,25223587,Crossing the Impassable: Genetic Connections i...,"Lessios, H. A.; Robertson, D. R.",2006,9,273,1598,2201-2208,dispersal; marine biogeography; Pacific Ocean;...,The 'impassable' Eastern Pacific Barrier (EPB)...,10.1098/rspb.2006.3652
1713,25249639,The Early Evolution of Feathers: Fossil Eviden...,"Perrichot, Vincent; Marion, Loïc; Néraudeau, D...",2008,5,275,1639,1197-1202,Feather evolution; Early stages; Cretaceous am...,The developmental stages of feathers are of ma...,10.1098/rspb.2008.0003
3510,30245016,Reversal of Evolutionary Downsizing Caused by ...,"Conover, David O.; Munch, Stephan B.; Arnott, ...",2009,6,276,1664,2015-2020,fisheries management; fishery-induced evolutio...,Evolutionary responses to the long-term exploi...,10.1098/rspb.2008.0003


In [27]:
# # save as .csv
# df.to_csv('data/metadata.csv.gz', index=False, sep='\t', compression='gzip')